# __오디오 파일을 받아쓰는 음성 인식 모델 만들기__

- 튜토리얼 난이도: ★☆☆☆☆
- 읽는데 걸리는 시간: 10분
- 사용 언어: [SQL](https://ko.wikipedia.org/wiki/SQL) (100%)
- 실행 파일 위치: tutorial/thanosql_ml/audio_recognition/speech_recognition.ipynb
- 참고 문서: [LibriSpeech 데이터 세트](http://www.openslr.org/12), [wav2vec 2.0: A Framework for Self-Supervised Learning of Speech Representations](https://arxiv.org/abs/2006.11477)

## 튜토리얼 소개

<div class="admonition note">
    <h4 class="admonition-title">음성 인식 기술 이해하기</h4>
    <p>컴퓨터 음성 인식 또는 음성-텍스트 변환(Speech-to-Text)이라고도 부르는 음성 인식 기술은 프로그램이 사람의 음성을 텍스트 형식으로 처리할 수 있도록 해줍니다. 최근 자동차, 의료 분야, 인공지능 스피커나 스마트폰을 이용한 일상 생활까지 광범위한 분야에서 활용되고 있습니다. 최근 <a href="https://ko.wikipedia.org/wiki/%EA%B8%B0%EA%B3%84_%ED%95%99%EC%8A%B5">머신러닝(기계학습/Machine Learning)</a> 알고리즘을 활용한 음성 인식 기술은 문법, 구문, 구조, 오디오와 음성 신호의 구성을 통합하여 음성을 이해하고 처리합니다.</p>
</div>

<div class="admonition warning">
    <p>일반적으로 음성 인식(Voice Recognition)과 혼동 될 수 있는데, 음성 인식은 개별 사용자의 목소리를 식별하는 데만 집중합니다.</p>
</div>

오늘날 음성 인식 기술은 다양한 산업 분야에서 응용되고 있습니다. 음성 인식 기술의 발전은 단순 여행용 자동 통역에서 난이도가 높은 비즈니스 회의 통역까지 확대되고 있습니다. 또한, 음성 인식 기술은 음성 합성 기술로 발전하여 가상 안내원이나 비서 역할을 수행하기도 하고 특정 연예인이나 유명인의 목소리를 흉내내어 정해진 지문을 목소리로 변환하기도 하며 응용되고 있습니다.

__아래는 ThanoSQL 음성 인식 모델의 활용 및 예시입니다.__

- 음성 인식 기술은 전화 상담 데이터를 텍스트로 변환하여 고객의 감정 분석이나 상담 트렌드 분석 등을 가능하게 합니다. 상담자는 음성 인식 기술을 통해, 고객의 문의에 대한 답변이 되는 정보나 과거의 유사 사례 등을 빠르게 제공받아 고객 상담을 개선할 수 있습니다.
또한, 상담 종료 후에는 음성으로 저장되어 있는 데이터를 활용하여 감정 분석을 통해 고객의 만족도 등을 간접적으로 측정하여 트렌드 별 고객의 만족도를 확인할 수 있습니다.
- 음성 인식 기술을 사용하면 키보드로 작성하는 메모보다 빠르게 작성이 가능하고, 긴 음성 파일에서도 빠르게 특정 키워드를 검색 할 수 있습니다.

<div class="admonition note">
    <h4 class="admonition-title">본 튜토리얼에서는</h4>
    <p>👉 Librispeech [Panayotov et al. 2015]는 음성인식 연구에 있어서 가장 널리 사용되는 대규모 영어 음성 데이터 중 하나로 사용자 참여형 오디오북 프로젝트인 <a href="https://librivox.org/">LibriVox project</a>의 결과물입니다. 16kHz로 샘플링된 약 1,000시간 분량의 녹음된 오디오북 데이터를 가공하여 만들었습니다. 튜토리얼을 위한 대상 테이블은 미리 업로드 된 음성 파일의 경로와 스크립트로 구성되어 있습니다. 본 튜토리얼은 음성 파일을 텍스트로 변환하는 것을 목표로 하고 있습니다.</p>
</div>

<div class="admonition warning">
    <h4 class="admonition-title">튜토리얼 주의 사항</h4>
    <ul>
        <li>ThanoSQL에서 현재 지원하는 오디오 파일의 형식은 '.wav', '.flac' 입니다.</li>
        <li>오디오 파일 경로를 나타내는 컬럼(Column)과 목푯값(Target)에 해당하는 텍스트를 나타내는 컬럼이 테이블에 존재해야 합니다.</li>
        <li>해당 음성 인식 모델의 베이스 모델(<strong>Wav2Vec2En</strong>)은 GPU를 사용합니다. 사용한 모델의 크기와 배치 사이즈에 따라 GPU 메모리가 부족할 수 있습니다. 이 경우, 더 작은 모델을 사용하시거나 배치 사이즈를 줄여보십시오.</li>
    </ul>
</div>

## __0. 데이터 세트 및 모델 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/ko/getting_started/paas/workspace/lab/)에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은_API_TOKEN>

### __데이터 세트 준비__

In [2]:
%%thanosql
GET THANOSQL DATASET librispeech_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" 쿼리 구문을 사용하여 원하는 데이터 세트를 워크스페이스에 저장합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL DATASET</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 데이터 세트가 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY librispeech_train
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/librispeech_data/librispeech_train.csv'

Success


In [4]:
%%thanosql
COPY librispeech_test
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/librispeech_data/librispeech_test.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>COPY</strong>" 쿼리 구문을 사용하여 데이터베이스에 저장 할 테이블명을 지정합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>COPY</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"if_exists": 동일 이름의 테이블이 존재하는 경우 처리하는 방법 설정. 오류 발생, 기존 테이블에 추가, 기존 테이블 대체 (str, optional, 'fail'|'replace'|'append', default: 'fail')</li>
        </ul>
        </li>
    </ul>
</div>

### __모델 준비__

In [5]:
%%thanosql
GET THANOSQL MODEL wav2vec2
OPTIONS (
    model_name='tutorial_audio_recognition',
    overwrite=True
    )

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL MODEL</strong>" 쿼리 구문을 사용하여 원하는 모델을 워크스페이스 및 데이터베이스에 저장합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL MODEL</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>“model_name”: 저장할 모델의 이름 (str, optional)</li>
            <li>"overwrite": 동일 이름의 모델이 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 모델은 새로운 모델로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

## __1. 데이터 세트 확인__

음성 인식 모델을 만들기 위해 ThanoSQL 워크스페이스 데이터베이스에 저장되어 있는 __librispeech_train__ 테이블을 사용합니다. 아래의 쿼리 구문을 실행하고 테이블의 내용을 확인합니다.

In [6]:
%%thanosql
SELECT *
FROM librispeech_train
LIMIT 5

,audio_path,text
0,thanosql-dataset/librispeech_data/000.wav,i noticed how white and well shaped his own ha...
1,thanosql-dataset/librispeech_data/001.wav,the only conflicts that occurred on irish soil...
2,thanosql-dataset/librispeech_data/002.wav,inquired shaggy in the metal forest
3,thanosql-dataset/librispeech_data/003.wav,my grandmother always spoke in a very loud ton...
4,thanosql-dataset/librispeech_data/004.wav,the poets of succeeding ages have dwelt much i...


<div class="admonition note">
    <h4 class="admonition-title">데이터 테이블 이해하기</h4>
    <p><strong>librispeech_train</strong> 테이블은 아래와 같은 정보를 담고 있습니다.</p>
    <ul>
        <li>audio_path: 음성 파일의 위치 경로</li>
        <li>text: 해당 음성의 목푯값 (target, 스크립트)</li>
    </ul>
</div>

In [7]:
%%thanosql
PRINT AUDIO
AS
SELECT audio_path
FROM librispeech_train
LIMIT 3

/home/jovyan/thanosql-dataset/librispeech_data/000.wav


/home/jovyan/thanosql-dataset/librispeech_data/001.wav


/home/jovyan/thanosql-dataset/librispeech_data/002.wav


## __2. 사전 학습된 모델을 사용하여 음성 인식 결과 예측__

다음 쿼리 구문을 실행하여 __tutorial_audio_recognition__ 모델을 사용하여 결과를 예측합니다.

In [8]:
%%thanosql
PREDICT USING tutorial_audio_recognition
OPTIONS (
    audio_col='audio_path',
    batch_size=8
    )
AS
SELECT *
FROM librispeech_train

,audio_path,text,predict_result
0,thanosql-dataset/librispeech_data/000.wav,i noticed how white and well shaped his own ha...,I NOTICED HOW WHITE AND WELL SHAPED HIS OWN HA...
1,thanosql-dataset/librispeech_data/001.wav,the only conflicts that occurred on irish soil...,THE ONLY CONFLICTS THAT OCCURRED ON IRISH SOIL...
2,thanosql-dataset/librispeech_data/002.wav,inquired shaggy in the metal forest,INQUIRED SHAGGY IN THE MEDAL FOREST
3,thanosql-dataset/librispeech_data/003.wav,my grandmother always spoke in a very loud ton...,MY GRANDMOTHER ALWAYS SPOKE IN A VERY LOUD TON...
4,thanosql-dataset/librispeech_data/004.wav,the poets of succeeding ages have dwelt much i...,THE POETS OF SUCCEEDING AGES HAVE DWELT MUCH I...
...,...,...,...
75,thanosql-dataset/librispeech_data/075.wav,we can't do anything without evidence complain,WE CAN'T DO ANYTHING WITHOUT EVIDENCE COMPLAIN
76,thanosql-dataset/librispeech_data/076.wav,when i came up he touched my shoulder and look...,WHEN I CAME UP HE TOUCHED MY SHOULDER AND LOOK...
77,thanosql-dataset/librispeech_data/077.wav,it relieved him for a while,IT RELIEVED HIM FOR A WHILE
78,thanosql-dataset/librispeech_data/078.wav,this world's thick vapours whelm your eyes unw...,THIS WORLD'S THICK VAPOURS WHELM YOUR EYES UNW...


## __3. 음성 인식 모델 생성__

이전 단계에서 확인한 __librispeech_train__ 테이블을 사용하여 음성 인식 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 <strong>my_speech_recognition_model</strong>이라는 이름의 모델을 만듭니다.  
(쿼리 실행 시 예상 소요 시간: 1 min)

In [9]:
%%thanosql
BUILD MODEL my_speech_recognition_model
USING Wav2Vec2En
OPTIONS (
    audio_col='audio_path',
    text_col='text',
    max_epochs=1,
    batch_size=4,
    overwrite= True
    )
AS
SELECT *
FROM librispeech_train

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" 쿼리 구문을 사용하여 <strong>my_speech_recognition_model</strong>이라는 모델을 만들고 학습시킵니다.</li>
        <li>"<strong>USING</strong>" 쿼리 구문을 통해 베이스 모델로 <strong>Wav2Vec2En</strong>을 사용할 것을 명시합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 모델 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"audio_col": 학습에 사용할 오디오의 경로가 기록되어 있는 컬럼의 이름 (str, default: 'audio_path')</li>
            <li>"text_col":  오디오의 스크립트 정보를 담은 컬럼의 이름 (str, default: 'text')</li>
            <li>"max_epochs": 모든 학습 데이터 세트를 학습하는 횟수 (int, optional, default: 5)</li>
            <li>"batch_size": 한 번의 학습에서 읽는 데이터 세트 묶음의 크기 (int, optional, default: 16)</li>
            <li>"overwrite": 동일 이름의 모델이 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 모델은 새로운 모델로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

<div class="admonition tip">
    <p>여기서는 빠르게 학습하기 위해 "max_epochs"를 1로 지정했습니다. 일반적으로 숫자가 클수록 많은 계산 시간이 소요되지만 학습이 진행됨에 따라 예측 성능이 올라갑니다.</p>
</div>

## __4. 생성된 모델을 사용하여 음성 인식 결과 예측__

이전 단계에서 생성한 음성 인식 모델을 사용해 특정 음성을 예측해 봅니다. 테스트용 데이터 세트(학습에 이용되지 않은 데이터 테이블, <strong>librispeech_test</strong>)를 사용합니다.

In [10]:
%%thanosql
PREDICT USING my_speech_recognition_model
OPTIONS (
    audio_col='audio_path',
    result_col='predict_result'
    )
AS
SELECT *
FROM librispeech_test

,audio_path,text,predict_result
0,thanosql-dataset/librispeech_data/080.wav,dead said doctor macklewain,DEAD SAID DOCTOR MACKELWAINE
1,thanosql-dataset/librispeech_data/081.wav,one day when i rode over to the shimerdas i fo...,ONE DAY WHEN I RODE OVER TO THE SHIMERIDASI FO...
2,thanosql-dataset/librispeech_data/082.wav,well i don't think you should turn a guy's t v...,WELL I DON'T THINK YOU SHOULD TURN A GUYSE TEE...
3,thanosql-dataset/librispeech_data/083.wav,and what allurements or what vantages upon the...,AND WHAT ALLUREMENTS OR WHAT VANTAGES UPON THE...
4,thanosql-dataset/librispeech_data/084.wav,yes how many,YE S HOW MANY
5,thanosql-dataset/librispeech_data/085.wav,then i look perhaps like what i am,THEN I LOOK PERHAPS LIKE WHAT I AM
6,thanosql-dataset/librispeech_data/086.wav,i'm mister christopher from london,I MISTER CHRISTOPHER FROM LONDON
7,thanosql-dataset/librispeech_data/087.wav,nature a difference of fifty years had set a p...,NATUREA DIFFERENCE OF FIFTY YEARSHAD SET A PRO...
8,thanosql-dataset/librispeech_data/088.wav,he is just married you know is he said burgess,HE IS JUST MARRIED YOU KNOWIS HE SAID BURGES
9,thanosql-dataset/librispeech_data/089.wav,she pointed into the gold cottonwood tree behi...,SHE POINTED IN TO THE GOLD COTTON WOOD TREE BE...


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>PREDICT USING</strong>" 쿼리 구문을 사용하여 이전 단계에서 만든 <strong>my_speech_recognition_model</strong> 모델을 예측에 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 예측에 사용할 옵션을 지정합니다.
        <ul>
            <li>"audio_col": 예측에 사용할 오디오의 경로가 기록되어 있는 컬럼의 이름 (str, default: 'audio_path')</li>
            <li>"result_col": 데이터 테이블에서 예측 결과를 담을 컬럼 이름 (str, optional, default: 'predict_result')</li>
        </ul>
        </li>
    </ul>
</div>

## __5. 튜토리얼을 마치며__

이번 튜토리얼에서는 LibriSpeech 데이터 세트를 사용하여 음성 인식 모델을 만들어 보았습니다. 초급 단계 튜토리얼인 만큼 정확도 향상을 위한 과정 설명보다는 작동 위주의 설명으로 진행했습니다. 음성 인식 모델은 각 플랫폼이나 서비스에 맞는 정밀한 튜닝을 통해 정확도를 향상 시킬 수 있습니다. 나만의 데이터를 이용해서 베이스 모델을 학습을 진행하고 성능을 향상시켜 보세요. 다양한 비정형 데이터(이미지, 비디오, 텍스트 등)와 수치형 데이터들을 결합하여 나만의 모델을 만들고 경쟁력있는 서비스를 만들 수 있습니다.

* [나만의 데이터 업로드하기](https://docs.thanosql.ai/ko/getting_started/data_upload/)
* [나만의 데이터 테이블 생성하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [나만의 모델 업로드하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/UPLOAD_MODEL_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">나만의 서비스를 위한 모델 배포 관련 문의</h4>
    <p>ThanoSQL을 활용해 나만의 모델을 만들거나, 나의 서비스에 적용하는데 어려움이 있다면 언제든 아래로 문의주세요😊</p>
    <p>음성 인식 모델 구축 관련 문의: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>